# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [46]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [2]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv('../Vacation_Search/WeatherPy_Vacation.csv')

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,7,Yenagoa,NG,77.34,few clouds,4.9247,6.2642,Peretimi Hotels Ltd.
1,15,Kalmunai,LK,75.78,broken clouds,7.4167,81.8167,Saji new food
2,21,Sinnamary,GF,77.43,overcast clouds,5.3833,-52.9500,Hôtel du fleuve
3,22,Vila Velha,BR,78.75,light rain,-20.3297,-40.2925,Hotel Prainha
4,28,Porto Novo,BJ,82.22,scattered clouds,6.4965,2.6036,Hôtel 6500 CFA


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [5]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Max Temp',
    scale = 1,
    color = 'City',
    hover_cols = ['Hotel Name', 'Country', 'Weather Description']
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Weather Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [6]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[(vacation_df['City'] == 'Maceio')]
vacation_end = vacation_df.loc[(vacation_df['City'] == 'Maceio')]
vacation_stop1 = vacation_df.loc[(vacation_df['City'] == 'Maragogi')]
vacation_stop2 = vacation_df.loc[(vacation_df['City'] == 'Tamandare')]
vacation_stop3 = vacation_df.loc[(vacation_df['City'] == 'Olinda')]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [8]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end])

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
100,391,Maceio,BR,78.24,clear sky,-9.6658,-35.7353,Maceió Hostel e Pousada
120,477,Maragogi,BR,79.20,few clouds,-9.0122,-35.2225,Pousada Mariluz
110,425,Tamandare,BR,79.39,scattered clouds,-8.7597,-35.1047,Pousada Baia dos Corais
16,54,Olinda,BR,82.13,scattered clouds,-8.0089,-34.8553,Fidji Motel
100,391,Maceio,BR,78.24,clear sky,-9.6658,-35.7353,Maceió Hostel e Pousada


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [11]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['Lat', 'Lng']].copy()

# Display sample data
waypoints_df

,Lat,Lng
100,-9.6658,-35.7353
120,-9.0122,-35.2225
110,-8.7597,-35.1047
16,-8.0089,-34.8553
100,-9.6658,-35.7353


## Use GeoViews to create map that shows the four cities in the itinerary

In [20]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 150,
    color = 'City',
    hover_cols = ['Hotel Name', 'Country', 'Weather Description']
)

In [21]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country,Weather Description)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [23]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [24]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '46bf6d63aca44960a8c3dfd706433bae',
 'waypoints': '-9.6658,-35.7353|-9.0122,-35.2225|-8.7597,-35.1047|-8.0089,-34.8553|-9.6658,-35.7353'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [31]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params).json()

# Convert the API response to JSON format
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-35.7353, -9.6658], 'original_index': 0},
     {'location': [-35.2225, -9.0122], 'original_index': 1},
     {'location': [-35.1047, -8.7597], 'original_index': 2},
     {'location': [-34.8553, -8.0089], 'original_index': 3},
     {'location': [-35.7353, -9.6658], 'original_index': 4}],
    'units': 'metric',
    'distance': 555856,
    'distance_units': 'meters',
    'time': 25395.232,
    'legs': [{'distance': 131172,
      'time': 6605.121,
      'steps': [{'from_index': 0,
        'to_index': 7,
        'distance': 460,
        'time': 34.84,
        'instruction': {'text': 'Drive northwest on Rua do Imperador.'}},
       {'from_index': 7,
        'to_index': 30,
        'distance': 742,
        'time': 56.869,
        'instruction': {'text': 'Turn right onto Ladeira do Brito.'}},
       {'from_index': 30,
        'to_index': 51,
        'distance': 916,
        'time': 47.463,
     

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [95]:
# Fetch the route's legs coordinates from the JSON reponse
leg1 = route_response['features'][0]['geometry']['coordinates'][0]
leg2 = route_response['features'][0]['geometry']['coordinates'][1]
leg3 = route_response['features'][0]['geometry']['coordinates'][2]
leg4 = route_response['features'][0]['geometry']['coordinates'][3]
leg1

[[-35.735359, -9.665836],
 [-35.73572, -9.665257],
 [-35.736371, -9.664185],
 [-35.737005, -9.663163],
 [-35.737073, -9.66304],
 [-35.737317, -9.66259],
 [-35.737325, -9.662573],
 [-35.737477, -9.662262],
 [-35.73733, -9.662191],
 [-35.736864, -9.66198],
 [-35.73678, -9.661942],
 [-35.736413, -9.661766],
 [-35.735509, -9.661167],
 [-35.735421, -9.661113],
 [-35.735028, -9.660873],
 [-35.734514, -9.660566],
 [-35.734357, -9.660438],
 [-35.73418, -9.660303],
 [-35.734007, -9.660196],
 [-35.733217, -9.65965],
 [-35.733155, -9.659569],
 [-35.733129, -9.659494],
 [-35.733127, -9.65947],
 [-35.733121, -9.65945],
 [-35.733117, -9.659429],
 [-35.733116, -9.659408],
 [-35.733159, -9.65929],
 [-35.733498, -9.658304],
 [-35.733506, -9.658251],
 [-35.733502, -9.658198],
 [-35.733452, -9.658031],
 [-35.733431, -9.657781],
 [-35.733343, -9.65675],
 [-35.733299, -9.656121],
 [-35.733227, -9.655349],
 [-35.733221, -9.655241],
 [-35.733199, -9.655019],
 [-35.733171, -9.654682],
 [-35.733146, -9.654206]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [96]:
# Create and empty list to store the longitude of each step
lng = []

# Create and empty list to store the latitude of step
lat = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for item in leg1:
    lng.append(item[0])
    lat.append(item[1])
for item in leg2:
    lng.append(item[0])
    lat.append(item[1])
for item in leg3:
    lng.append(item[0])
    lat.append(item[1])
for item in leg4:
    lng.append(item[0])
    lat.append(item[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [97]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df['longitude'] = lng
route_df['latitude'] = lat
# Display sample data
route_df.head()

,longitude,latitude
0,-35.735359,-9.665836
1,-35.735720,-9.665257
2,-35.736371,-9.664185
3,-35.737005,-9.663163
4,-35.737073,-9.663040


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [98]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    frame_width=700,
    frame_height=500,
    line_width=3,
    color='black'
)

In [99]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country,Weather Description)
   .Path.I   :Path   [longitude,latitude]